# Konvoluční sítě v PyTorch

**Pozn.:** Modul `ans.py` se musí nacházet buď ve stejném adresáři, ze kterého je spuštěný tento notebook, nebo v aresáři, jehož cesta je zapsaná v vystémové proměnné PYTHONPATH.

Před supštěním příkazu `jupyter notebook` tedy nastavte:
- na Windows: `set PYTHONPATH=c:\path\to\ans-repo;%PYTHONPATH%`
- na Linuxu: `export PYTHONPATH=/path/to/ans-repo:$PYTHONPATH`

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tqdm

## Konvoluce v PyTorch pomocí vrstvy

V PyTorch jsou vrstvy různých typů definované v modulu `nn`. Např. konvoluční vrstva je definována jako třída `Conv2d` obalující funkci `torch.nn.functional.conv2d`, přičemž si uchovává parametry `weight`, `bias` a další. V implementaci dopředného průchodu vrstvy `nn.Conv2d` najdeme:

``` python
class Conv2d(_ConvNd):
    r"""Applies a 2D convolution over an input signal composed of several input
    planes.    
    ...
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True):
        # ... inicializace a dalsi
    
    def forward(self, input):
        return F.conv2d(input, self.weight, self.bias, self.stride, 
            self.padding, self.dilation, self.groups
```

Podobně jsou definovány i ostatní vrstvy, jako např. lineární atd. Rozdílem oproti manuální implementaci z minulých cvičení je absence definice zpětného průchodu. Ten je definován na nižší úrovni kódu v C++ v knihovně ATen, na níž PyTorch staví. O jeho zavolání ve správný čas ve zpětné propoagaci se postará PyTorch modul autograd. Např. jiná knihovna [Chainer](https://chainer.org/) je naopak stavěná tak, že zpětný průchod je definován explicitně přímo v Pythonu u každé operace, a to jak pro CPU, tak pro GPU (CUDA). Jde tedy o podobný způsob jako u cvičení [multilayer-perceptron](multilayer-perceptron.ipynb). Výhodou může být, že u každé vrstvy lze krokováním projít výpočet gradientů na jednotlivé parametry a vstupy.

Konvoluční vrstvu v PyTorch vytvoříme jako objekt typu `nn.Conv2d`, jehož kontruktor po nás chce počet výstupních kanálů konvoluce (=počet filtrů), počet vstupních kanálů (=3 pro RGB), velikost filtru (=3 pro filtr 3x3), a zda chceme přičítat i bias.

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

In [ ]:
conv = nn.Conv2d(3, 1, 3, padding=1, bias=False)
conv

PyTorch všechny vrstvy při vytvoření defaultně inicializuje. Vlastní nastavení je možné, např.
``` python
nn.init.xavier_uniform(conv.weight)
```
nebo přímo kopírováním do `conv.weight` metodou `copy_`.

Před aplikací vrstvy nejprve načteme obrázek.

In [ ]:
rgb_test = cv2.imread('data/happy-green-frog.jpg')[..., ::-1]
rgb_test.dtype, rgb_test.shape, rgb_test.min(), rgb_test.max()

In [ ]:
plt.imshow(rgb_test)

Všechny vrstvy očekávají 4D vstup ve formátu *dávka x kanály x výška x šířka* a defaultně jako typ `float32`.

In [ ]:
input_tensor = rgb_test.transpose(2, 0, 1)  # prevod na kanaly x vyska x sirka
input_tensor = input_tensor[None]  # navic dimenze davky
input_tensor = torch.from_numpy(input_tensor / 255.).float() # prevod do PyTorch a float32 v rozsahu 0..1
input_tensor.dtype, input_tensor.shape

Dopředný průchod už je jednoduchý:

In [ ]:
conv_output = conv(input_tensor)
conv_output.shape

## 2D max pooling

Jak jsme si ukazovali na přednášce, konvoluce v neurosítích se obvykle používá ve spojení s max poolingem, který na na každém typicky např. 2x2 okně počítá maximum. Na rozdíl od konvoluce se okénka typicky nepřekrývají a výstup je tedy menší než vstup, např. poloviční pro 2x2 max pool okénko.

V PyTorchi je max pooling implementován třídou `torch.nn.MaxPool2d`, která obaluje funkci `torch.nn.functional.max_pool2d`.

In [ ]:
# kernel_size = 2, stride = None --> 2x2 okenko s krokem 2
pool = nn.MaxPool2d(2)
pool

In [ ]:
pool_output = pool(conv_output)
pool_output.shape

## Definice konvoluční sítě

### Pomocí `torch.nn.Sequential`

PyTorch umožňuje více způsobů definice vlastní neuronové sítě. Nejjednodušším je pomocí třídy `torch.nn.Sequential`, do níž zadáme seznam vrstev, jak jdou za sebou. Tím je zároveň definován dopředný průchod, který proto nemusíme ručně implementovat.

In [ ]:
seq = nn.Sequential(
    nn.Conv2d(3, 1, 3, padding=1, bias=False),
    nn.MaxPool2d(2)
)
seq

Následující kód provede dopředný průchod konvolucí a poté max-pooling stejně, jako jsme to udělali nahoře.

In [ ]:
seq_output = seq(input_tensor)
seq_output.shape

### Pomocí třídy odvozené z `torch.nn.Module`

Třída `Sequential` je samozřejmě limitovaná na jednoduché sítě, v nichž každá vrstva má právě jeden vstup a jeden výstup. Neumožňuje tak definovat např. residuální bloky. Druhým, oficiálním a doporučeným postupem je proto definice vlastní třídy odvozené od `torch.nn.Module`. Tento způsob je sice poněkud "ukecanější", ale zato mnohem flexibilnější. Stejný model jako pomocí třídy `Sequential` zadefinujeme následovně.

In [ ]:
class Conv(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv = nn.Conv2d(3, 1, 3, padding=1, bias=False)
        self.pool = nn.MaxPool2d(2)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.pool(x)
        return x

In [ ]:
conv = Conv()
conv

In [ ]:
conv_output = conv(input_tensor)
conv_output.shape

### Kombinovaná síť

Vytvoříme rovněž kombinovanou konvoluční síť pro klasifikaci. Po konvolučních vrstvách bude následovat plně propojená vrstva produkující skóre pro jednotlivé třídy tak, jak to známe z minýlch úloh. Všimněme si, že lineární vrstva potřebuje znát přesnou velikost výstupu předchozí vrstvy. Jelikož budeme pracovat s obrázky 32x32 a dvakrát zmenšíme na polovinu max-poolingem, poslední konv. mapa bude mít rozměr 8x8.

**Pozn.:** jelikož vrstvy jako např. max pooling či ReLU nemají žádné trénovatelné parametry, často se jako objekty typu `nn.Module` neinstancují a pouze se aplikují jako funkce v dopředném průchodu. Viz konstruktor a metodu `forward`.

In [ ]:
class Convnet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1, bias=False)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1, bias=False)
        self.fc1 = nn.Linear(32 * 8 * 8, 64)
        self.fc2 = nn.Linear(64, 10)
    
    def forward(self, x):
        # prvni konv. vrstva
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        # druha konv. vrstva
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x

In [ ]:
model = Convnet()
model

Pokud chceme použít GPU, zavoláme z objektu typu `nn.Module` metodu `.to()`, která za parametr přijímá datový typ nebo zařízení, do kterého se má model převést. Funkce funguje "in-place", tzn. že modifikuje daný objekt, byť na něj zároveň vrací referenci. Následující buňka proto bude mít výstup.

In [ ]:
model.to('cuda')

### Parametry sítě

Modely v PyTorch umožňují procházet seznam svých parametrů metodami `parameters()` nebo `named_parameters()`. To je důležité např. pro ukládání naučených sítí. Funkce si přitom rekurzivně poradí i s vnořenými modely. To znamená, že výsledný seznam získaný metodou `net.named_parameters()` našeho modelu `Net` obsahuje parametry všech jeho atributů, které jsou typu odvozeného z `torch.nn.Module`.

In [ ]:
for name, par in model.named_parameters():
    print(name, par.dtype, par.shape, par.numel())

## Automatický zpětný průchod v Pytorch: autograd

Knihovna Pytorch umožňuje automatickou derivaci (autograd) a zpětnou propagaci gradientu následujícím způsobem. Pokud vytvoříme jakýkoliv `Tensor` s dodatečným parametrem `requires_grad=True`, knihovna si zapamatuje všechny operace s ním provedené. Viz příklad přímo z [pytorch.org](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html).

In [ ]:
x = torch.ones(2, 2, requires_grad=True)
x

Pokud s `x` něco provedeme, autograd si to zapamatuje.

In [ ]:
y = x + 2
y

Další dvě operace:

In [ ]:
z = y * y * 3
z

In [ ]:
w = z.mean()
w

PyTorch autograd si nyní pamatuje celou historii výpočtů od `x` až po `w`. Na sled výpočtů můžeme nahlížet jako na výpočetní graf, v němž uzly představují jednotlivé proměnné a matematickou funkci, pomocí které jejich hodnota byla vypočtena. Pokud nyní na `w` zavoláme metodu `backward(dout)` s nějakým "příchozím" gradientem `dout` (defaultně je `None`), spustí se kompletní zpětná propagace celým výpočetním stromem až k "listu" `x`.

In [ ]:
w.backward()

Zpětný průchod v PyTorch má tedy jiné API než to, co jsme vytvořili v úloze [multilayer-perceptron](multilayer-perceptron.ipynb). Metoda `backward` zde nevrací gradienty na parametry dané vrstvy, ale spustí plnou zpětnou propagaci z její pozice ve výpočetním grafu až ke vstupu. Vypočtené gradienty se přitom úkládají přímo do uzlů tvořených jednotlivými proměnnými, do jejich atributu `grad`.

Např. derivace `w` podle `x`, tj. gradient $dw / dx$ je:

In [ ]:
x.grad

Ovšem pokud bychom se chtěli podívat na $dw / dz$:

In [ ]:
z.grad

`z.grad` je `None`, protože PyTorch z důvodů šetření paměti zahazuje mezivýpočty, tj. všechny gradienty, které netvoří list stromu. Pokud bychom ho přesto chtěli vidět, lze na `z` zavolat funkci `retain_grad()` a znovu spustit zpětný průchod. K tomu však musíme znovu vytvořit celý výpočetní graf, jelikož PyTorch po zavolání backpropu defaultně vyčistí jeho buffery (toto chování lze změnit argumentem `retain_graph` metody `backward`).

In [ ]:
y = x + 2
z = y * y * 3
z.retain_grad()
w = z.mean()
w.backward()
z.grad

Podívejme se znovu na gradient na `x`.

In [ ]:
x.grad

Ačkoliv jsme provedli stejné výpočty, gradient na `x` se změnil! Stalo se tak proto, že autograd ve skutečnosti atribut `grad` nepřepisuje, ale při každém backpropu akumuluje. Tensor `x` totiž nebyl znovu vytvořen, nýbrž byl použit dvakrát a výsledný gradient je proto součtem příchozích gradientů ze dvou pod-stromů.

Pokud bychom vytvořili i `x` znovu, reference na původní `x` by přestala existovat, z prvního výpočetního grafu by nezbylo již nic a gradient by byl stejný jako při prvním průchodu. Dokud však proměnná s "cachovaným" `grad` existuje, není možné vyčistit paměť, což může být v PyTorch zdrojem nepříjemných bugů a zaplňování paměti.

## Kritérium

Zda budeme optimalizovat křížovou entropii (Softmax) nebo hinge loss (SVM) zvolíme vytvořením odpovídajícího objektu. V PyTorch jsou nejčastěji používaná kritéria reprezentovaná jako vrstvy typu `nn.Module` s dopředným průchodem definovaným metodou `forward`, jehož výstup je typicky jediné číslo repezentující hodnotu lossu.

In [ ]:
crit = nn.CrossEntropyLoss()
crit

## Metoda optimalizace parametrů

PyTorch umožňuje trénování pomocí celé řady variant Stochastic Gradient Descentu jako např. momentum SGD, RMSprop, Adam a další. Optimalizátor je objekt, který převezme seznam parametrů, jež má optimalizovat, a výchozí learning rate. Objektu předáme parametry naší `Convnet` a nastavíme learning rate.

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer

Stěžejní funkcionalitou `optimizer`u je metoda `step`, která updatuje všechny tensory zadané v seznamu parametrů v konstruktoru. Tuto funkci voláme po výpočtu gradientů zpětným průchodem v každé minidávce. V případě `SGD` bez momentu se provede `param -= learning_rate * dparam`, jak to známe z minulých úloh.

## Práce s daty

Pytorch unsadňuje práci s daty pomocí předdefinovaných datasetů a loaderů, které umožňují procházení po dávkách. V našem případě pracujeme s CIFAR-10, pro který existuje třída `torchvision.datasets.CIFAR10`, kterou již známe z minulých úloh.

In [ ]:
import torchvision
from torchvision import transforms

Pro snadnější přípravu dat a případnou agumentaci obsahuje balík torchvision modul `transforms`. Základní třídou obalující funkcionalitu řetezce transformací je `Compose`. Při konstrukci přebírá `list` transformací, které budeme na obrázky aplikovat, např. normalizace na definovanou velikost, odečtení průmeru apod. Jelikož všechny obrázky v CIFAR-10 jsou stejně velké, použijeme zde pouze třídu `ToTensor`, která převede obrázek ze třídy `PIL.Image` na `torch.Tensor` a zároveň s tím normalizuje do rozsahu 0..1 a přeskupí dimenze do formátu *kanály x výška x šířka* tak, jak to konvoluce v PyTorch vyžaduje.

*Pozn.:* Transformace `ToTensor` je nutná pro zpracování po dávkách. Pokud bychom ji nepoužili, PyTorch by nevěděl, jak má objekty typu `PIL.Image` seskupit do jedné dávky, tj. souvislého 4D tensoru. Ostatní transformace však pracují právě s `PIL.Image`, proto jí vždy umístíme až na konec.

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()
])

Vytvořený objekt předáme do konstruktoru datasetu jako jeden z parametrů.

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_dataset

In [ ]:
train_dataset.classes

In [ ]:
valid_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
valid_dataset

### Procházení dat

Načítání dat po dávkách zajišťuje třída `torch.utils.data.DataLoader`. Do konstruktoru jí předáme objekt datasetu, požadovanou velikost dávky a příp. informaci, zda se mají obrázky vybírat náhodně či postupně za sebou tak, jak jsou seřazeny v datasetu.

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=64, shuffle=False)

Přes výsledný objekt je možné iterovat a v každém kroku vrátí dvojici `X, y` obrázků a jejich labelů.

In [ ]:
inputs, targets = next(iter(train_loader))
inputs.dtype, inputs.shape, targets.dtype, targets.shape

Pokud používáme GPU, defaultně jsou data na procesoru.

In [ ]:
next(model.parameters()).device == inputs.device

Stejně jako s modelem, převod na GPU zajistíme zavoláním metody `to('cuda')`.

In [ ]:
next(model.parameters()).device == inputs.to('cuda').device

Rozdílem ovšem je, že operace `.to()` nad `torch.Tensor` není "in-place" a nemodifikuje tak původní data, nýbrž vytváří jejich kopii.

In [ ]:
inputs.device

## Trénovací cyklus

Nyní jsou všechny komponenty potřebné pro trénování připravné, stačí tedy sestavit je za sebe do trénovacího cyklu. Ten bude velmi podobný jako v minulých úlohách a jeho vnitřní krok je implementován metodou `train_step`. Opět se jedná o standardní cyklus, kde se opakuje dopředný průchod, výpočet lossu, zpětný průchod a update parametrů.

In [ ]:
import ans

Jako první je nutné přepnout model do trénovacího režimu zavoláním `model.train()`. Toto je důležité především kvůli vrstvám jako je `Dropout` nebo `BatchNorm2d`, které se při trénování chovají jinak než při validaci. Např. batch normalizace ve fázi inference již nepočítá střední hodnotu a standardní odchylku, ale namísto toho použije hodnoty vypočtené v trénování. Funkce `model.train()` přepne interní příznak `training` všech atributů objektu `model`, které jsou třídy `torch.nn.Module`, na hodnotu `True`. Jinými slovy, všechny child moduly `model`u jako např. `conv1` budou mít tento příznak nastaven na `True`.

In [ ]:
model.training, model.conv1.training

In [ ]:
model.train()

In [ ]:
model.training, model.conv1.training

Jako další pošleme metodou `.to(device)` data (obrázky a jejich anotace) na zařízení, na němž se `model` nachází. Zdali je model na cpu či gpu zjistíme např. tak, že se podíváme na některý z jeho parametrů. Samotná třída `torch.nn.Module` tuto informaci neuchovává.

In [ ]:
device = next(model.parameters()).device  # model.parameters() je generator
device

In [ ]:
inputs, targets = inputs.to(device), targets.to(device)
inputs.device, targets.device

Nyní můžeme provést dopředný průchod modelem. Zavolat lze buď metodu `model.forward(inputs)`, ale obvyklejší je zkrácená verze `model(inputs)`. Třída `torch.nn.Module` má totiž definovaný operátor `()`, který `.forward()` zavolá za nás. Všechny objekty vrstev typu `torch.nn.Module` proto vypadají a chovají se jako funkce.

In [ ]:
scores = model(inputs)
scores.dtype, scores.shape

Posledním krokem dopředného průchodu je výpočet hodnoty lossu. Protože jsme zvolili křížovou entropii implementovanou třídou `torch.nn.CrossEntropyLoss`, pošleme do výpočtu lineární skóre `scores` a anotace `targets`.

In [ ]:
loss = crit(scores, targets)
loss

V tuto chvíli `loss` v sobě obsahuje celou historii výpočtu od `inputs` a `targets` až po výslednou hodnotu. Celá tato historie je reprezentována jako výpočetní graf, jenž známe z přednášek. Graf musí mít stromovou strukturu a nesmí obsahovat cykly. Na jeho základě jsou poté vypočteny metodou zpětné propagace gradienty na jednotlivé listy, z nichž některé představují parametry modelu a jiné zase vstupy (`inputs` a `targets`). Gradient interních uzlů, tj. mezivýpočtů, jako jsou např. `scores`, se z důvodů paměťové optimalizace defaultně neuchovávají.

Celá zpětná propagace se spustí zavoláním `loss.backward()`. Po jejím skončení bude mít každý list stromu (terminální uzel grafu) vyplněn atribut `.grad`, v němž bude hodnota gradientu na proměnnou reprezentovanou tímto uzlem.

In [ ]:
loss.backward()

In [ ]:
model.conv1.weight.grad

**Ovšem pozor!** Jak bylo uvedeno výše, metoda `.backward()` atributy `.grad` nepřepisuje, ale **přičítá** k nim. Pokud bychom proto zavolali zpětnou propagaci ještě jednou, hodnoty v `.grad` se zdvojnásobí. PyTorch nám však neumožní jednoduše znovu zavolat `loss.backward()`, protože s jeho zavoláním zároveň vyčití paměť výpočetního grafu. Nepředpokládá totiž, že bychom potřebovali volat zpětný průchod 2x. Omezení pro demonstrační účely přičítání gradientů obejdeme tak, že stejný dopředný průchod zavoláme ještě jednou.

In [ ]:
crit(model(inputs), targets).backward()

In [ ]:
model.conv1.weight.grad

Všimněme si, že gradienty se opravdu sečetly a hodnoty jsou nyní dvojnásobné oproti prvnímu zpětnému průchodu. Abychom tomuto chování zabránili, je nutné před každým zpětným průchodem zavolat `optimizer.zero_grad()`, která projde všechny trénované parametry, které jsme u `optimizer`u zaregistrovali při vytváření, a vyčistí u nich `.grad` na nuly. Tento krok je naprosto nezbytný, jelikož v opačném případě nám v nové dávce (batch) zůstanou staré gradienty z minula a síť se nebude učit správně.

In [ ]:
optimizer.zero_grad()

In [ ]:
model.conv1.weight.grad

In [ ]:
# jelikoz jsme si nyni gradienty vynulovali, jeste jednou jeste spocitame, abychom mohli provest update parametru
crit(model(inputs), targets).backward()

Update parametrů zajistí metoda `.step()` třídy `torch.optim.Optimizer`. Projde všechny své registrované parametry a provede u nich update dle algoritmu, který implementuje. V našem přípdě jsme zvolili `torch.optim.SGD`, provede se tedy jednoduchý update tvaru $\theta_{t+1} := \theta_t - \gamma\nabla L(\theta_t)$, kde $\theta_t$ reprezentuje všechny parametry našeho modelu a $\gamma$ značí krok učení (learning rate).

In [ ]:
model.conv1.weight

In [ ]:
optimizer.step()

In [ ]:
model.conv1.weight

Opakováním uvedených kroků s různými dávkami vstupů a anotací z datasetu CIFAR10 můžeme natrénovat libovolnou neuronovou síť v úloze klasifikace obrázků, viz notebook [conv-classification](conv-classification.ipynb).